In [74]:
import json

import requests
import pandas as pd
import numpy as np

from datetime import timedelta

from dotenv import load_dotenv

from config import Config
from duration_rounder import DurationRounder
from errors import RequestError
from report import REPORT_DETAIL_URL, get_request_params, get_auth, create_datarow, calc_rounded_hours

load_dotenv(verbose=False)

config = Config('configs/bounteous.json')

response = requests.get(REPORT_DETAIL_URL, params=get_request_params(), auth=get_auth())

if not response.ok:
    raise RequestError.create(response)

In [75]:
json_data = json.dumps(response.json()['data'])
dataframe = pd.read_json(json_data, convert_dates=['start', 'end', 'updated'])
dataframe['start'] = dataframe['start'].dt.tz_localize(None)
dataframe['date'] = dataframe['start'].dt.to_period('D')
dataframe['duration'] = dataframe['dur'].apply(lambda x: timedelta(milliseconds=x))
dataframe = dataframe[['date', 'client', 'project', 'description', 'duration']]
dataframe = dataframe.rename(columns={'description': 'task'})

In [76]:
dataframe = dataframe.groupby(['date', 'client', 'project', 'task'], as_index=False).sum()
dataframe = dataframe.sort_values(by=['date', 'client', 'project', 'task'])
dataframe

,date,client,project,task,duration
0,2020-03-09,B8-Leave/Absence,Vacation,Vacation day,08:00:00
1,2020-03-10,B8-Firm Administration,Administration,Client Info Share,01:03:16
2,2020-03-10,RN-DFF/RTN Enhancements,Coding / Development,Amex merchant registration code review,02:14:50
3,2020-03-10,RN-DFF/RTN Enhancements,Coding / Development,Troubleshoot Visa merchant update issue,00:18:17
4,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Card Team Standup,00:14:01
5,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Rewards Network checkin,00:19:31
6,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Rewards Network/Mastercard Weekly Production M...,00:29:45
7,2020-03-10,RN-DFF/RTN Enhancements,QA Testing,CI-591: Test Jesse White changes for Visa,01:57:33
8,2020-03-10,SS-Core Team 2020,Back - End Development,Review project documentation / architecture re...,01:27:20
9,2020-03-11,RN-DFF Amex,Coding / Development,CI-601: Batch commits to Kafka for amex-mids-sync,00:25:24


In [77]:
duration_rounder = DurationRounder(config.round_to_minutes())
dataframe['rounded_duration'] = dataframe['duration'].apply(duration_rounder.round)
dataframe['rounded_hours'] = dataframe['rounded_duration'].apply(calc_rounded_hours)
dataframe

,date,client,project,task,duration,rounded_duration,rounded_hours
0,2020-03-09,B8-Leave/Absence,Vacation,Vacation day,08:00:00,08:00:00,8.00
1,2020-03-10,B8-Firm Administration,Administration,Client Info Share,01:03:16,01:05:00,1.08
2,2020-03-10,RN-DFF/RTN Enhancements,Coding / Development,Amex merchant registration code review,02:14:50,02:15:00,2.25
3,2020-03-10,RN-DFF/RTN Enhancements,Coding / Development,Troubleshoot Visa merchant update issue,00:18:17,00:20:00,0.33
4,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Card Team Standup,00:14:01,00:15:00,0.25
5,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Rewards Network checkin,00:19:31,00:20:00,0.33
6,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Rewards Network/Mastercard Weekly Production M...,00:29:45,00:30:00,0.50
7,2020-03-10,RN-DFF/RTN Enhancements,QA Testing,CI-591: Test Jesse White changes for Visa,01:57:33,02:00:00,2.00
8,2020-03-10,SS-Core Team 2020,Back - End Development,Review project documentation / architecture re...,01:27:20,01:25:00,1.42
9,2020-03-11,RN-DFF Amex,Coding / Development,CI-601: Batch commits to Kafka for amex-mids-sync,00:25:24,00:25:00,0.42


In [78]:
daily_totals = dataframe.groupby('date', as_index=False).sum()
daily_totals['client'] = np.nan
daily_totals['project'] = np.nan
daily_totals['task'] = np.nan
daily_totals

,date,client,project,task,duration,rounded_duration,rounded_hours
0,2020-03-09,NaN,NaN,NaN,08:00:00,08:00:00,8.00
1,2020-03-10,NaN,NaN,NaN,08:04:33,08:10:00,8.16
2,2020-03-11,NaN,NaN,NaN,08:19:35,08:20:00,8.34
3,2020-03-12,NaN,NaN,NaN,08:20:10,08:15:00,8.25
4,2020-03-13,NaN,NaN,NaN,07:55:47,08:00:00,8.00


In [79]:
combined_df = pd.concat([dataframe, daily_totals])
combined_df = combined_df.sort_values(by=['date', 'client', 'project', 'task'], na_position='last')
combined_df['client'] = combined_df['client'].fillna('')
combined_df['project'] = combined_df['project'].fillna('')
combined_df['task'] = combined_df['task'].fillna('')
combined_df

,date,client,project,task,duration,rounded_duration,rounded_hours
0,2020-03-09,B8-Leave/Absence,Vacation,Vacation day,08:00:00,08:00:00,8.00
0,2020-03-09,,,,08:00:00,08:00:00,8.00
1,2020-03-10,B8-Firm Administration,Administration,Client Info Share,01:03:16,01:05:00,1.08
2,2020-03-10,RN-DFF/RTN Enhancements,Coding / Development,Amex merchant registration code review,02:14:50,02:15:00,2.25
3,2020-03-10,RN-DFF/RTN Enhancements,Coding / Development,Troubleshoot Visa merchant update issue,00:18:17,00:20:00,0.33
4,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Card Team Standup,00:14:01,00:15:00,0.25
5,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Rewards Network checkin,00:19:31,00:20:00,0.33
6,2020-03-10,RN-DFF/RTN Enhancements,Meetings / BDD / Story Mapping,Rewards Network/Mastercard Weekly Production M...,00:29:45,00:30:00,0.50
7,2020-03-10,RN-DFF/RTN Enhancements,QA Testing,CI-591: Test Jesse White changes for Visa,01:57:33,02:00:00,2.00
8,2020-03-10,SS-Core Team 2020,Back - End Development,Review project documentation / architecture re...,01:27:20,01:25:00,1.42
